In [8]:
# Import
import torch
import torch.nn as nn
import numpy as np
from dataloader import BatchDataloader
from tqdm.notebook import trange, tqdm
import h5py
from torch.utils.data import TensorDataset, random_split, DataLoader
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
from utils import pgd_attack, eval_loop, eval_loop_apgd, filter_adversarial
from models import ResNet1dGELU
import ecg_plot
from utils import plot_ecgs
import ast
%matplotlib inline

In [14]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tqdm.write("Use device: {device:}\n".format(device=device))

Use device: cuda



In [15]:
# hyperparameters
batch_size = 32

In [16]:
# Load the data
dataset_path = '/local_storage/users/arveri/code-15'

path_to_csv = dataset_path + '/exams.csv'
path_to_val = dataset_path + '/test.h5'
path_to_train = dataset_path + '/train10.h5'

# Get labels
df = pd.read_csv(path_to_csv, index_col='exam_id')

# Get h5 file
train_file = h5py.File(path_to_train, 'r')
train_traces_ids = train_file['exam_id']

val_file = h5py.File(path_to_val, 'r')
val_traces_ids = val_file['exam_id']

# Only keep the traces in the csv that match the traces in the h5 file
train_df = df[df.index.isin(train_traces_ids)]
val_df = df[df.index.isin(val_traces_ids)]

# Define traces
train_traces = train_file['tracings']
val_traces = val_file['tracings']

# Sort the dataframe in trace order
train_df = train_df.reindex(train_traces_ids)
val_df = val_df.reindex(val_traces_ids)

# Get labels
train_labels = train_df['age'].values
val_labels = val_df['age'].values

# Only use x% of the training data
train_traces = train_traces#[:len(train_traces)//div]
train_labels = train_labels#[:len(train_labels)//div]

# Count each label
unique, counts = np.unique(val_labels, return_counts=True)

# Print it
print(dict(zip(unique, counts)))

unique_train, counts_train = np.unique(train_labels, return_counts=True)
print(dict(zip(unique_train, counts_train)))

# Print size of training and validation set
print(f"Training set size: {len(train_labels)}")
print(f"Validation set size: {len(val_labels)}")

# Make into torch tensor
train_labels = torch.tensor(train_labels, dtype=torch.float32).reshape(-1,1)
val_labels = torch.tensor(val_labels, dtype=torch.float32).reshape(-1,1)

# Define dataloaders
train_dataloader = BatchDataloader(train_traces, train_labels, batch_size=batch_size)
val_dataloader = BatchDataloader(val_traces, val_labels, batch_size=batch_size)

{0: 13892, 1: 20577}


In [ ]:
n_boot = 500

In [17]:
loss_function = nn.MSELoss()

# Create dictionary for storing metrics for each model
metrics = {}

for model_path in ["../models/age/ptb_20_apgd_1e-2_code15_lr1e-4_div1/latest.pth", "../models/age/ptb_20_apgd_5e-2_code15_lr1e-4_div1/latest.pth", "../models/age/ptb_20_code15_lr1e-4_div1/latest.pth"]:
    # Define the model
    model = ResNet1dGELU(input_dim=(12, 4096),n_classes=1, blocks_dim=[(64, 4096), (128, 1024), (196, 256), (256, 64), (320, 16)])#, kernel_size=3, dropout_rate=0.8
    checkpoint = torch.load(model_path, map_location=lambda storage, loc: storage)
    model.load_state_dict(checkpoint['model'])
    model.to(device)
    
    metrics[model_path] = {}
    
    valid_loss, y_pred, y_true = eval_loop(0, val_dataloader, model, loss_function, device)
    
    # save y_pred and y_true to same directory as model
    path = model_path.split('.pth')[0]
    
    with open(path + '_y_pred.npy', 'wb') as f:
        np.save(f, y_pred)
    with open(path + '_y_true.npy', 'wb') as f:
        np.save(f, y_true)

    # bootstrap!!!

    # def bootstrap(y_test, y_pred, metric, quantiles, n_boot=500):
    bootstrapped_maes = np.zeros(n_boot)
    bootstrapped_mses = np.zeros(n_boot)
    
    for i in range(n_boot):
        indices = np.random.choice(range(len(y_pred)), len(y_pred))
        bootstrapped_maes[i] = mean_absolute_error(y_true[indices], y_pred[indices])
        bootstrapped_mses[i] = mean_squared_error(y_true[indices], y_pred[indices])

    mae_q1 = np.quantile(bootstrapped_maes, 0.25)
    mae_q3 = np.quantile(bootstrapped_maes, 0.75)
    mae_med = np.median(bootstrapped_maes)

    mse_q1 = np.quantile(bootstrapped_mses, 0.25)
    mse_q3 = np.quantile(bootstrapped_mses, 0.75)
    mse_med = np.median(bootstrapped_mses)

    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    
    # store the metrics for this model
    metrics[model_path] = {'mae': mae, 'mse': mse, 'mae_q1': mae_q1, 'mae_q3': mae_q3, 'mae_med': mae_med, 'mse_q1': mse_q1, 'mse_q3': mse_q3, 'mse_med': mse_med}

Evaluation Epoch  0:   0%|          | 0/1078 [00:00<?, ?it/s]

Evaluation Epoch  0:   0%|          | 0/1078 [00:00<?, ?it/s]

Evaluation Epoch  0:   0%|          | 0/1078 [00:00<?, ?it/s]

In [18]:
# Print table comparing results
from tabulate import tabulate

table = []

for model_path, model_metrics in metrics.items():
    table.append([model_path, model_metrics['mae'], model_metrics['mae_q1'], model_metrics['mae_q3'], model_metrics['mae_med'], model_metrics['mse'], model_metrics['mse_q1'], model_metrics['mse_q3'], model_metrics['mse_med']])
    
print(tabulate(table, headers=['Model', 'MAE', 'MAE Q1', 'MAE Q3', 'MAE Median', 'MSE', 'MSE Q1', 'MSE Q3', 'MSE Median']))


Model                                                       Accuracy     AUROC        AP    Max F1
--------------------------------------------------------  ----------  --------  --------  --------
../models/sex/ptb_20_code15ft_div10/latest.pth              0.781949  0.849451  0.869388  0.832831
../models/sex/ptb_20_apgd_5e-2_code15ft_div10/latest.pth    0.767356  0.837207  0.872097  0.820666
../models/sex/ptb_20_apgd_1e-2_code15ft_div10/latest.pth    0.778148  0.850438  0.884768  0.828249


In [7]:
loss_function = nn.BCEWithLogitsLoss()

# Create dictionary for storing metrics for each model
metrics = {}

# Test for different epsilon values
eps_list = [0, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5]

for model_path in ["models/model_latest_20_fine_tunedlr1e-3/latest.pth"]:
    # Define the model
    model = ResNet1dGELU(input_dim=(12, 4096),n_classes=1, blocks_dim=[(64, 4096), (128, 1024), (196, 256), (256, 64), (320, 16)])#, kernel_size=3, dropout_rate=0.8
    checkpoint = torch.load(model_path, map_location=lambda storage, loc: storage)
    model.load_state_dict(checkpoint['model'])
    model.to(device)
    
    metrics[model_path] = {}
    
    for eps in eps_list:
        # Evaluate with adversarial examples
        adv_valid_loss, adv_y_pred, adv_y_true = eval_loop_apgd(0, val_dataloader, model, loss_function, device, adversarial=True, adv_eps=eps, adv_iters=10, adv_restarts=1)
        
        # apply sigmoid to y_pred
        adv_y_pred = torch.sigmoid(torch.tensor(adv_y_pred)).numpy()
            
        adv_auroc = roc_auc_score(adv_y_true, adv_y_pred)
        adv_ap = average_precision_score(adv_y_true, adv_y_pred)
        
        # compute max F1 score (over threshold)
        thresholds = np.linspace(0, 1, 100)
        f1_scores = [f1_score(adv_y_true, adv_y_pred > threshold, average='binary') for threshold in thresholds]
        best_threshold = thresholds[np.argmax(f1_scores)]
        best_f1 = np.max(f1_scores)
        
        adv_y_pred = np.round(adv_y_pred)

        # compute accuracy
        adv_accuracy = accuracy_score(adv_y_true, adv_y_pred)
        adv_f1 = f1_score(adv_y_true, adv_y_pred, average='binary')
        
        # store the metrics for this model
        metrics[model_path][eps] = {'accuracy': adv_accuracy, 'auroc': adv_auroc, 'ap': adv_ap, 'max f1': best_f1}

FileNotFoundError: [Errno 2] No such file or directory: 'models/model_latest_20_fine_tunedlr1e-3/latest.pth'

In [ ]:
import json

# Save these metrics to a file
for model_path, model_metrics in metrics.items():
    save_path = model_path.replace('.pth', '_apgd_eval.json')
    with open(save_path, 'w') as f:
        json.dump(model_metrics, f)    

In [ ]:
# Load model metrics 
import json

loaded_metrics = {}
for model_path in ['models/code_model_10/latest.pth', 'models/model_latest_adv_20_0.01exp_apgd_fine_tunedlr1e-3/latest.pth', 'models/model_latest_adv_20_0.01exp_apgd_fine_tunedlr1e-3ep20/latest.pth']:
    with open(model_path.replace('.pth', '_apgd_eval.json'), 'r') as f:
        loaded_metrics[model_path] = json.load(f)

In [ ]:
import utils
from importlib import reload
import autopgd_base
from matplotlib.ticker import AutoMinorLocator
import matplotlib as mpl
plt.style.use(['style.mpl'])

# plot the accuracy for each model and each epsilon
labels = ['model 1', 'model 2', 'model 3']
colors = ['blue', 'red', 'green']
linestyles = ['-', '--', '-.']
eps_list = [0, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5]

for model_path, color, linestyle, label in zip(loaded_metrics, colors, linestyles, labels):
    # get the metrics for this model
    model_metrics = loaded_metrics[model_path]
    
    # get the accuracy for each epsilon
    acc = [model_metrics[eps]['ap'] for eps in model_metrics]
    
    # plot the accuracy
    plt.plot([str(eps) for eps in eps_list], acc, label=label, marker='o', linestyle=linestyle, linewidth=1, color=color)
    
plt.grid()

# change figure size
fig = plt.gcf()
fig.set_size_inches(6, 4)

# make it higher resolution
mpl.rcParams['figure.dpi'] = 300

# Set font size
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# set remaining fonts such as axis and legend
plt.legend(fontsize=12)
plt.xlabel('$\epsilon$', fontsize=12)
plt.ylabel('AUPRC', fontsize=12)

plt.ylim(0, 1)

plt.show()

In [ ]:
# plot the f1 score for each model and each epsilon
for model_path in metrics:
    # get the metrics for this model
    model_metrics = metrics[model_path]
    
    # get the accuracy for each epsilon
    f1 = [model_metrics[eps]['f1'] for eps in model_metrics]
    
    # plot the accuracy
    plt.plot([str(eps) for eps in eps_list], f1, label=model_path, marker='o')
    
plt.xlabel('Epsilon')
plt.ylabel('F1 score')
plt.grid()

plt.ylim(0, 1)
plt.legend()

plt.show()

In [ ]:
model = ResNet1dGELU(input_dim=(12, 4096),n_classes=1, blocks_dim=[(64, 4096), (128, 1024), (196, 256), (256, 64), (320, 16)])#, kernel_size=3, dropout_rate=0.8
checkpoint = torch.load("model_latest_adv_20_5e-2exp.pth", map_location=lambda storage, loc: storage)
model.load_state_dict(checkpoint['model'])
model = model.to(device)
print("Model loaded!")

In [ ]:
import utils
from importlib import reload
import autopgd_base

reload(utils)
reload(autopgd_base)

eps = 5e-2
cuttoff_freq = 150

# Plot some adversarial examples
traces, diagnoses = next(iter(val_dataloader))

ecg_sample = np.transpose(traces[0])

traces = traces.to(device)
diagnoses = diagnoses.to(device)

# TEST APGD
# import apgd
# attack = apgd.APGD(model, norm='Linf', eps=1e-3, steps=10, n_restarts=1, seed=0, loss='bce', eot_iter=1, rho=.75, verbose=False)
# traces = attack(traces, diagnoses)
diagnoses = diagnoses.reshape(-1)


print(traces.shape)
print(diagnoses.shape)

attack = autopgd_base.APGDAttack(model, n_iter=20, norm='Linf', n_restarts=1, eps=eps, seed=0, loss='ce', eot_iter=1, rho=.75)
attack.init_hyperparam(traces)
attack.attack_single_run(traces, diagnoses)


traces = pgd_attack(model, traces, diagnoses, device, eps=eps, alpha=eps/5, steps=10)

ecg_sample_adv = np.transpose(traces[0].cpu().numpy())

# Filter it
ecg_sample_adv = filter_adversarial(ecg_sample_adv, sample_rate=400, fc=cuttoff_freq)


# Select the first lead
ecg_sample = ecg_sample[0:1]

ecg_sample_adv = ecg_sample_adv[0:1]

plt.figure()
#lead = ['I', 'II', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'I-adv', 'II-adv', 'V1-adv', 'V2-adv', 'V3-adv', 'V4-adv', 'V5-adv', 'V6-adv']
lead = ['I', 'I-adv', 'II', 'II-adv', 'V1', 'V1-adv', 'V2', 'V2-adv', 'V3', 'V3-adv', 'V4', 'V4-adv', 'V5', 'V5-adv', 'V6', 'V6-adv']
#lead = ['I', 'I-adv']
utils.plot_ecgs(ecg_sample, ecg_sample_adv, sample_rate=400, style = 'bw', row_height=6, lead_index=lead, columns=1, title="",show_zoom=True, zoom_box=[3.0, 4.0, -0.2, 0.3], zoom_rate=10)
plt.legend(['Adversarial', 'Original'])
plt.show()


In [ ]:
eps = 5e-2
loss_function = nn.BCEWithLogitsLoss()

fcs = [1, 2, 5, 10, 20, 50, 75, 100, 125, 150]

# Create dictionary for storing metrics for each model
metrics = {}

for model_path in ['model_latest_20.pth', "model_latest_adv_20_5e-2exp.pth"]:
    # Define the model
    model = ResNet1dGELU(input_dim=(12, 4096),n_classes=1, blocks_dim=[(64, 4096), (128, 1024), (196, 256), (256, 64), (320, 16)])#, kernel_size=3, dropout_rate=0.8
    checkpoint = torch.load(model_path, map_location=lambda storage, loc: storage)
    model.load_state_dict(checkpoint['model'])
    model.to(device)
    
    metrics[model_path] = {}
    
    for fc in fcs:
        # Evaluate with adversarial examples
        adv_valid_loss, adv_y_pred, adv_y_true = eval_loop(0, val_dataloader, model, loss_function, device, adversarial=True, adv_eps=eps, adv_alpha=eps/5, adv_steps=10, post_process=filter_adversarial, post_process_args=[400, fc])

        adv_auroc = roc_auc_score(adv_y_true, adv_y_pred)
        adv_ap = average_precision_score(adv_y_true, adv_y_pred)

        # apply sigmoid to y_pred
        adv_y_pred = torch.sigmoid(torch.tensor(adv_y_pred)).numpy()
        adv_y_pred = np.round(adv_y_pred)

        # compute accuracy
        adv_accuracy = accuracy_score(adv_y_true, adv_y_pred)
        adv_f1 = f1_score(adv_y_true, adv_y_pred, average='binary')
        
        # store the metrics for this model
        metrics[model_path][fc] = {'accuracy': adv_accuracy, 'auroc': adv_auroc, 'ap': adv_ap, 'f1': adv_f1}

In [ ]:
# plot the ap for each model and each epsilon
for model_path in metrics:
    # get the metrics for this model
    model_metrics = metrics[model_path]
    
    # get the accuracy for each epsilon
    ap = [model_metrics[fc]['ap'] for fc in model_metrics]
    
    # plot the accuracy
    plt.plot([str(fc) for fc in fcs], ap, label=model_path, marker='o')

plt.xlabel('Cutoff frequency')
plt.ylabel('AUPRC')
plt.grid()

plt.ylim(0, 1)
plt.legend()

plt.show()
